In [1]:
from __future__ import division
from __future__ import print_function
from operator import itemgetter
from itertools import combinations, chain
import time
import datetime
import os
import tensorflow as tf
import numpy as np
import networkx as nx
import scipy.sparse as sp
from sklearn import metrics
import pandas as pd
import psutil
import pickle
from decagon.deep.optimizer import DecagonOptimizer
from decagon.deep.model import DecagonModel
from decagon.deep.minibatch import EdgeMinibatchIterator
from decagon.utility import rank_metrics, preprocessing

In [4]:
in_file = './data/data_structures/DECAGON/DECAGON_toy_genes_500_drugs_400_se_4'
with open(in_file, 'rb') as f:
    DS = pickle.load(f)
    for key in DS.keys():
        globals()[key]=DS[key]
        print(key,"Imported successfully")

edge2name Imported successfully
se_mono_name2idx Imported successfully
gene2idx Imported successfully
nonzero_feat Imported successfully
edge_type2dim Imported successfully
adj_mats_orig Imported successfully
edge_type2decoder Imported successfully
se_combo_name2idx Imported successfully
drug2idx Imported successfully
degrees Imported successfully
edge_types Imported successfully
num_edge_types Imported successfully
num_feat Imported successfully
feat Imported successfully


In [2]:
opti = 'data/data_structures/intento_optimizer'
with open(opti, 'rb') as f:
    opt = pickle.load(f)

In [5]:
np.shape(opt[1,1][1])

(400, 400)

In [3]:
mb_file = 'data/data_structures/MINIBATCH/MINIBATCH_toy_genes_500_drugs_400_se_4_batchsize_128_valsize_0.15_noise_0.05'
with open(mb_file, 'rb') as f:
    minibatch = pickle.load(f)

In [ ]:
def get_accuracy_scores(edges_pos, edges_neg, edge_type, rec, noise=False):
    """ Returns the AUROC, AUPRC and Accuracy of the dataset corresponding to the edge
    'edge_type' given as a tuple. The parameters 'edges_pos' and 'edges_neg' are the list 
    of edges of positive and negative interactions respectively of a given dataset, i.e., 
    train, test or validation.
    """
    # Predict on set of edges
    preds = []
    for u, v in edges_pos:
        score = sigmoid(rec[u, v])
        preds.append(score)
        if not noise:
            assert adj_mats_orig[edge_type[:2]][edge_type[2]][u,v] > 0, 'Problem 1'
    preds_neg = []
    for u, v in edges_neg:
        score = sigmoid(rec[u, v])
        preds_neg.append(score)
        if not noise:
            assert adj_mats_orig[edge_type[:2]][edge_type[2]][u,v] == 0, 'Problem 0'
    preds_all = np.hstack([preds, preds_neg])
    preds_all = np.nan_to_num(preds_all)
    labels_all = np.hstack([np.ones(len(preds)), np.zeros(len(preds_neg))])

    roc_sc = metrics.roc_auc_score(labels_all, preds_all)
    aupr_sc = metrics.average_precision_score(labels_all, preds_all)
    acc = metrics.accuracy_score(labels_all, np.round(preds_all))

    return roc_sc, aupr_sc, acc